In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import array
from pickle import dump
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import LambdaCallback, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
from keras.utils import to_categorical
import tensorflow as tf
import string

Using TensorFlow backend.
/Users/neilwatt/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [4]:
# load text to memory
def load_doc(filename): 
# open the file as read only
    file = open(filename, 'r')
# read all text
    text = file.read()
# close the file
    file.close()
    return text

In [5]:

##heuristics for splitting up words
#replace dashes with a white space
#otherwise words based on white space.
#reduce all words to lowercase and remove punctuation from words 
#remove all words that are not alphabetic 
# turn a doc into clean tokens



def clean_doc(doc):
# replace '--' with a space ' '
    doc = doc.replace('--', ' ')
# split into tokens by white space
    tokens = doc.split()
# remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
# make lower case
    tokens = [word.lower() for word in tokens]
    return tokens


In [6]:
# use GPU to speed up training time through tensforflow
config = tf.ConfigProto()
#only allocate as much GPU memory based on runtime allocations, initially little but allows memory to be extended
config.gpu_options.allow_growth = True


In [7]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()


In [8]:
# load document
in_filename = '/Users/neilwatt/Documents/BIs/PrWeb/2018Posts/October/canterburytales.txt'
doc = load_doc(in_filename)
print(doc[:200])

The Project Gutenberg Etext of The Canterbury Tales and Other Poems
by Geoffrey Chaucer






                      THE CANTERBURY TALES
                         And other Poems
                      


In [9]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))


['the', 'project', 'gutenberg', 'etext', 'of', 'the', 'canterbury', 'tales', 'and', 'other', 'poems', 'by', 'geoffrey', 'chaucer', 'the', 'canterbury', 'tales', 'and', 'other', 'poems', 'of', 'geoffrey', 'chaucer', 'edited', 'for', 'popular', 'perusal', 'by', 'd', 'laing', 'purves', 'contents', 'preface', 'life', 'of', 'chaucer', 'the', 'canterbury', 'tales', 'the', 'general', 'prologue', 'the', 'knights', 'tale', 'the', 'millers', 'tale', 'the', 'reeves', 'tale', 'the', 'cooks', 'tale', 'the', 'man', 'of', 'laws', 'tale', 'the', 'wife', 'of', 'baths', 'tale', 'the', 'friars', 'tale', 'the', 'sompnours', 'tale', 'the', 'clerks', 'tale', 'the', 'merchants', 'tale', 'the', 'squires', 'tale', 'the', 'franklins', 'tale', 'the', 'doctors', 'tale', 'the', 'pardoners', 'tale', 'the', 'shipmans', 'tale', 'the', 'prioresss', 'tale', 'chaucers', 'tale', 'of', 'sir', 'thopas', 'chaucers', 'tale', 'of', 'meliboeus', 'the', 'monks', 'tale', 'the', 'nuns', 'priests', 'tale', 'the', 'second', 'nuns',

In [16]:
(set(tokens))

{'carried',
 'bodekins',
 'carthage',
 'jollily',
 'galley',
 'circumstances',
 'avicenna',
 'weenest',
 'guessing',
 'authors',
 'unthank',
 'physiologus',
 'nation',
 'pal',
 'snubnosed',
 'renounce',
 'disposd',
 'readily',
 'blunder',
 'ordered',
 'arras',
 'debased',
 'whip',
 'abandoned',
 'tercel',
 'abie',
 'dishonesty',
 'wore',
 'appaird',
 'spiced',
 'quench',
 'amulius',
 'rey',
 'countenance',
 'katharos',
 'gauren',
 'princples',
 'second',
 'duchess',
 'skilful',
 'loseth',
 'tended',
 'shaft',
 'making',
 'yshewd',
 'lade',
 'invisible',
 'magistrate',
 'proudest',
 'deliverly',
 'england',
 'caitiffs',
 'richness',
 'pore',
 'tonight',
 'troubadours',
 'signe',
 'zech',
 'waileth',
 'fate',
 'visage',
 'se',
 'stot',
 'showed',
 'gnawing',
 'ourselves',
 'forlete',
 'tooles',
 'ronian',
 'holyday',
 'paraventure',
 'present',
 'navarre',
 'leaveless',
 'bookes',
 'orpheus',
 'seedfowl',
 'pound',
 'sinnes',
 'stark',
 'mandements',
 'merite',
 'waist',
 'gnof',
 'defen

In [32]:
def map_book(tokens):
    hash_map = {}

    if tokens is not None:
        for element in tokens:
            # Remove Punctuation
            word = element.replace(",","")
            word = word.replace(".","")

            # Word Exist?
            if word in hash_map:
                hash_map[word] = hash_map[word] + 1
            else:
                hash_map[word] = 1

        return hash_map
    else:
        return None

In [33]:
word_list = [set(tokens)]


[{'carried',
  'bodekins',
  'carthage',
  'jollily',
  'galley',
  'circumstances',
  'avicenna',
  'weenest',
  'guessing',
  'authors',
  'unthank',
  'physiologus',
  'nation',
  'pal',
  'snubnosed',
  'renounce',
  'disposd',
  'readily',
  'blunder',
  'ordered',
  'arras',
  'debased',
  'whip',
  'abandoned',
  'tercel',
  'abie',
  'dishonesty',
  'wore',
  'appaird',
  'spiced',
  'quench',
  'amulius',
  'rey',
  'countenance',
  'katharos',
  'gauren',
  'princples',
  'second',
  'duchess',
  'skilful',
  'loseth',
  'tended',
  'shaft',
  'making',
  'yshewd',
  'lade',
  'invisible',
  'magistrate',
  'proudest',
  'deliverly',
  'england',
  'caitiffs',
  'richness',
  'pore',
  'tonight',
  'troubadours',
  'signe',
  'zech',
  'waileth',
  'fate',
  'visage',
  'se',
  'stot',
  'showed',
  'gnawing',
  'ourselves',
  'forlete',
  'tooles',
  'ronian',
  'holyday',
  'paraventure',
  'present',
  'navarre',
  'leaveless',
  'bookes',
  'orpheus',
  'seedfowl',
  'pou

In [34]:
hash_of_words = map_book(tokens)
hash_of_words 

{'carried': 12,
 'unskilful': 1,
 'affirmed': 3,
 'carthage': 8,
 'galley': 1,
 'circumstances': 14,
 'avicenna': 1,
 'bodekins': 2,
 'authors': 11,
 'plea': 1,
 'unthank': 1,
 'peculiarity': 1,
 'nation': 3,
 'pal': 1,
 'snubnosed': 1,
 'renounce': 4,
 'disposd': 2,
 'readily': 5,
 'blunder': 1,
 'debased': 1,
 'jollily': 2,
 'whip': 3,
 'abandoned': 2,
 'tercel': 7,
 'abie': 4,
 'dishonesty': 1,
 'wore': 10,
 'spiced': 4,
 'quench': 2,
 'eyne': 4,
 'fee': 2,
 'chapiter': 1,
 'rey': 1,
 'countenance': 43,
 'gauren': 3,
 'princples': 1,
 'loseth': 1,
 'second': 50,
 'dictys': 1,
 'reveste': 1,
 'shaft': 2,
 'making': 24,
 'yshewd': 1,
 'fatherland': 1,
 'invisible': 3,
 'magistrate': 1,
 'proudest': 2,
 'sometimes': 41,
 'deliverly': 1,
 'england': 11,
 'rocketh': 1,
 'richness': 1,
 'pore': 8,
 'cecilias': 1,
 'tonight': 6,
 'troubadours': 1,
 'signe': 6,
 'waileth': 3,
 'visage': 27,
 'weenest': 1,
 'se': 2,
 'stot': 1,
 'greeting': 2,
 'showed': 15,
 'powders': 2,
 'gnawing': 1,
 'o

In [35]:
#dwords=pd.DataFrame(hash_of_words.items(), columns=['Word', 'Count'])
#dwords

dwords = pd.DataFrame(list(hash_of_words.items()), columns=['Word', 'Count'])
dwords

,Word,Count
0,carried,12
1,unskilful,1
2,affirmed,3
3,carthage,8
4,galley,1
5,circumstances,14
6,avicenna,1
7,bodekins,2
8,authors,11
9,plea,1


In [40]:
#create delete words array
#find that 8220 words appeared only one
#in other words, 46% of the training set consists of single use words!
deletedf=dwords[(dwords['Count'] < 2)]

delete_list=deletedf['Word'].values
delete_list

array(['unskilful', 'galley', 'avicenna', ..., 'yelling', 'treasons',
       'cordial'], dtype=object)

In [42]:
#delete all words on delete list from corpus and 


#remove word Chapter
fin = open("/Users/neilwatt/Documents/BIs/PrWeb/2018Posts/October/canterburytales.txt", encoding='utf-8')
fout = open("canterburytales_Clean.txt", "w+", encoding='utf-8')

for line in fin:
    for word in delete_list:
        line = line.replace(word, "")
    fout.write(line)
fin.close()
fout.close()



In [ ]:
# organize into sequences of tokens
#difference between sequences (previous) and sentences is that I fixed issue
#with repeating words
length = 50 + 1
#move forward 3 words
step = 2
sequences = list()
for i in range(length, len(tokens), step):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

In [ ]:
# save sequences to file
out_filename = 'canterburytales_sequences.txt'

save_doc(sequences, out_filename)

In [ ]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'canterburytales_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [ ]:
#representation of the model is distributed
#- i.e words with similar meaning have similar representation
#representation learned simulataneously with model
#probability of next word using context of last 100 words
#embedding layer for representation, LSTM to predict words on context

# integer encode sequences of words
#mapping each word to unique integer and encoding input sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
#i add a one since vocabulary begins from index one 
#but indexing arrays is zero offset
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
tokenizer.word_index

In [ ]:

#need to seperate word encodings into  y (output) and x (input)
# separate output from input with array slicing
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
Embedding

In [ ]:

# defining model
# two LSTM hidden layers with 100 memory cells
#dense fully connected layer with 100 neurons connects to the LSTM hidden layers 
#to interpret the features extracted from the sequence
#output layer predicts the next word as a single vector the size of the vocabulary 
#with a probability for each word in the vocabulary
#softmax activation function is used to ensure 
#the outputs have the characteristics of normalized probabilities
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100))
#model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
#model.add(LSTM(100))
#model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(100))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

In [ ]:

# compile model
optimizer = RMSprop(lr=0.007)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# define the checkpoint so I can load model in future
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                 
                             mode='min')
earlystop = EarlyStopping(monitor='val_loss', patience=2,
                              verbose=2)

# fit model using the gpu
#batch size is a particularly important hyperparamter which I intend to play around within future posts
with tf.device('/gpu:0'):

    history=model.fit(X, y,validation_split=0.1,batch_size=80,epochs=60,verbose=1,callbacks=[ checkpoint,earlystop])


In [ ]:
print(history.history.keys())

In [ ]:
history.history['val_loss']

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 


In [ ]:
n_words

In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)
 
# load cleaned text sequences
in_filename = '/Users/neilwatt/Documents/BIs/PrWeb/2018Posts/October/Code/canterburytales_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('weights.hdf5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 3)
print(generated)

In [ ]:
seed_text